# Download Active Listings
Etsy API limits the offset to 12,000 listings, so that is the maximum number of recently updated listings that can be accessed. (Requested 100 at a time because each request is limited to 100 records)

In [1]:
import requests
import pandas as pd
import json

Get all listings that match keywords "stoneware" and "handmade"

In [2]:
#GetListings URL and Parameters for the API
url = "https://openapi.etsy.com/v3/application/listings/active"
key = "r8ymjsnfohuidr1wev7e4cfg"
keyword = "stoneware,handmade"

In [3]:
#initialize the dataframes
dataframe = pd.DataFrame()
df = pd.DataFrame()

In [4]:
#loop that increases the offset by 100 each time due to the API limitation of downloading a maximum of 100 listings at a time
limit = 100
offset = 0
for i in range (1,121):
    parameters = {"client_id":key, "limit":limit, "offset":offset, "keywords":keyword}
    download = requests.get(url, params=parameters)
    data = download.json()
    if "error" in data.keys():
        print("Maximum Offset Reached")
    else:
        dataframe = pd.json_normalize(data['results'])
        df = df.append(dataframe, ignore_index = True)
    offset = offset + limit
    i = i+1
    

In [5]:
#confirm that there are 12,000 rows
len(df)

12000

In [6]:
#write the data to a csv file
#I'm just using this as a backup to save the file in case the next loop doesnt work
#df.to_csv('12000listingsStonewareHandmade033122.csv')

## Download Shop Info for all the Etsy listings in the data



In [7]:
#NOTE: DELETE LATER You dont need this if you are doing downloading the 12,000 listings and the shop info at the same time
#df = pd.read_csv('12000listingsStonewareHandmade032222.csv')

In [8]:
#Get all the unique shop IDs
shop_ids = df["shop_id"].tolist()
shop_list = list(set(shop_ids))

In [9]:
? shop_list

In [10]:
#initialize the dataframes
dataframeShop = pd.DataFrame()
dfShop = pd.DataFrame()

# Get Shop Info
This loop sends a request to the ETSY API to get the shop info (ratings, number of favorers, etc) for each shop that appeared in the previous dataset

In [11]:
#loop that queries the API for 
limit = 100

for j in range(0, len(shop_list)):
    shop_id = str(shop_list[j])
    url = "https://openapi.etsy.com/v3/application/shops/" + shop_id
    parameters = {"client_id":key, "limit":limit}
    download = requests.get(url, params=parameters)
    dataShops = download.json()
    dataframeShop = pd.json_normalize(dataShops)
    dfShop = dfShop.append(dataframeShop, ignore_index = True)

In [12]:
dfShop

,shop_id,shop_name,user_id,create_date,title,announcement,currency_code,is_vacation,vacation_message,sale_message,...,policy_shipping,policy_refunds,policy_additional,policy_seller_info,policy_update_date,policy_has_private_receipt_info,has_unstructured_policies,policy_privacy,review_average,review_count
0,7266306,MalenkaDesign,24635635,1347577309,Handcrafted fonctional pottery from Santa Moni...,instagram: malenkadesign_ceramics\r\n\r\nI loo...,USD,False,None,None,...,I ship via USPS First Class Mail. \r\n\r\n****...,In the unlikely event your pottery is damaged ...,If you are sending your purchase as a gift for...,None,1423758137,False,True,None,5,13
1,19709955,EversArtShop,73391987,1553216295,Spreading love through art!,Hello my friends and welcome to EversArtShop w...,USD,False,None,None,...,PROCESSING TIME\nThe time I need to prepare an...,"I GLADLY ACCEPT RETURNS, EXCHANGES, AND CANCEL...",,None,0,False,False,,5,20
2,26124291,FoxCreekCeramics,317688587,1605750569,Hand-thrown ceramics in stoneware and English ...,Welcome to my site. I am a small boutique pot...,USD,False,Hello and thanks for visiting! I’m working on ...,Thanks so much for making a purchase from Fox ...,...,PROCESSING TIME\nThe time I need to prepare an...,I GLADLY ACCEPT RETURNS\nJust contact me withi...,- I do not share your contact information with...,None,0,False,False,I do not share your contact information with a...,5,1
3,32661510,Atelierreverberecrea,555114053,1645633184,None,None,EUR,False,Hello and thanks for visiting! I’m working on ...,None,...,None,None,None,None,0,False,False,None,None,None
4,20561927,xiaochengmei,229898845,1564765895,Mark&#39;s Studio,Welcome to Mark&#39;s Studio! \r\nPlease rea...,USD,False,None,None,...,PROCESSING TIME\nThe time I need to prepare an...,I GLADLY ACCEPT RETURNS AND EXCHANGES\nJust co...,,None,0,False,False,,4.899,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2735,22544373,THESTELLART,281352825,1593955936,"A shop where magic meets creativity, an online...",None,EUR,False,None,None,...,PROCESSING TIME\nThe time I need to prepare an...,I GLADLY ACCEPT RETURNS\nJust contact me withi...,,None,0,False,False,,5,18
2736,16556021,TheSecondBackPorch,20660948,1511986501,Collection of Previously Loved Treasures,Welcome to The Second Back Porch featuring an ...,USD,False,The Second Back Porch is on a short break and ...,Thank you for your purchase from The Second Ba...,...,PROCESSING TIME\nThe time I need to prepare an...,I GLADLY ACCEPT RETURNS\nJust contact me withi...,ALL PREVIOUSLY LOVED TREASURES IN MY SHOP . . ...,None,0,False,False,,5,1
2737,5169140,ReneeRodgersPottery,5145116,1179245602,Renee Rodgers Pottery,Welcome to Renee&#39; Rodgers Pottery. This is...,USD,False,"Welcome to my shop! Browse around, and check b...",Thank you for shopping Renee&#39; Rodgers Pott...,...,Shipping will occur on Saturday after transact...,None,None,None,1627433559,False,True,None,5,11
2738,32096248,LoveoftheClay,535595918,1646001865,Handmade Stoneware Pottery,FREE SHIPPING on orders $35 or more\nNew Easte...,USD,False,None,None,...,PROCESSING TIME\nThe time I need to prepare an...,I GLADLY ACCEPT CANCELLATIONS\nRequest a cance...,IS THE POTTERY FOOD-SAFE?\nYES. All glazes use...,None,0,False,False,,5,4


In [13]:
#write the data to a csv file
#I'm just using this as a backup to save the file in case the merge doesnt work
#df.to_csv('ShopInfo033122.csv')

## Merge the data

In [14]:
#create a dataframe with just the columns I'm interested in for the listings data
listings = df[['listing_id', 'shop_id', 'title',
       'description', 'num_favorers',
       'listing_type', 'tags', 'materials',
       'style','taxonomy_id','price.amount',
       'price.divisor', 'price.currency_code','creation_timestamp', 'ending_timestamp',
       'original_creation_timestamp','quantity']]

In [15]:
#create a dataframe with just the columns I'm interested in for the shops data
shops = dfShop[['shop_id', 'shop_name', 'create_date', 'title',
       'announcement', 'currency_code', 'is_vacation', 'vacation_message',
       'sale_message',
       'listing_active_count',
       'accepts_custom_requests', 'url',
       'num_favorers', 'languages', 'is_shop_us_based', 'transaction_sold_count',
       'shipping_from_country_iso', 'shop_location_country_iso',
       'review_average', 'review_count']]

In [16]:
#merge the two datasets so that the shop information is added to each listing
data = listings.merge(shops, on="shop_id")

In [19]:
# save the merged data
#NOTE: change the date each time (maybe automate the filename with the correct date for the future)

#data.to_csv('mergedData033122.csv')